In [ ]:
import glob
import os
import numpy as np
import cv2
import pandas    

readpath = 'E:/IP/test/*.tif'
labels = 'E:/IP/Labels/test.csv'

images = glob.glob(readpath)
labelfile = open(labels,'w')

for image in images:
    labelfile.write(image+'\n')

labelfile.close()

df = pandas.read_csv(labels)

rows = df.iterrows()
index= []
X= []

for row in rows:
    image = row[1][0]
    img = cv2.imread(image)
    #get the name of image
    indx= os.path.basename(image)
    img = cv2.resize(img,(126,126))
    img=np.asarray(img)
    imgRes=img/255 
    X.append(imgRes)
    #appending the image name in index array
    index.append(indx)
 
x_test=np.asarray(X)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras import backend as K
K.set_image_data_format('channels_last')

seed = 7
np.random.seed(seed)
num_classes =10


def CNN_model():
    model = Sequential()
    model.add(Convolution2D(64, 3, 3, border_mode= 'valid', input_shape=(126,126,3),activation= 'relu' ))
    model.add(Convolution2D(32, 3, 3, activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten()) 
    model.add(Dense(128, activation= 'relu' ))
    model.add(Dense(num_classes, activation= 'softmax' )) 
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model


model = CNN_model()
model.load_weights('best.hdf5')

dict = {1:'HTC-1-M7',2:'iPhone-4s',3:'iPhone-6',4:'LG-Nexus-5x',5:'Motorola-Droid-Maxx',6:'Motorola-Nexus-6',7:'Motorola-X',8:'Samsung-Galaxy-Note3',9:'Samsung-Galaxy-S4',10:'Sony-NEX-7'}
    
predicts= []

for i in range(2640):
    y = model.predict_classes(x_test[i])
    classno = np.ndarray.tolist(y)
    objectclass = dict[classno[0]]
    predicts.append(objectclass)
       
df = pd.DataFrame(columns=['fname', 'camera'])
df['fname'] = index
df['camera'] = predicts
df.to_csv("sub.csv", index=False)